In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point
import numpy as np
import pandas_profiling
from geopy.distance import great_circle
import numpy as np
import datetime

<b>File Descriptions</b><br><https://www.kaggle.com/c/predict-west-nile-virus/data>
<p><b>train.csv, test.csv</b> - the training and test set of the main dataset. The training set consists of data from 2007, 2009, 2011, and 2013, while in the test set you are requested to predict the test results for 2008, 2010, 2012, and 2014.</p>
<ul>
    <li><b>Id:</b> the id of the record</li>
<li><b>Date:</b> date that the WNV test is performed</li>
<li><b>Address:</b> approximate address of the location of trap. This is used to send to the GeoCoder.</li> 
<li><b>Species:</b> the species of mosquitos</li>
<li><b>Block:</b> block number of address</li>
<li><b>Street:</b> street name</li>
<li><b>Trap:</b> Id of the trap</li>
<li><b>AddressNumberAndStreet:</b> approximate address returned from GeoCoder</li>
<li><b>Latitude, Longitude:</b> Latitude and Longitude returned from GeoCoder</li>
<li><b>AddressAccuracy:</b> accuracy returned from GeoCoder</li>
<li><b>NumMosquitos:</b> number of mosquitoes caught in this trap</li>
<li><b>WnvPresent:</b> whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present.</li>
    </ul>
<p><b>spray.csv</b> - GIS data of spraying efforts in 2011 and 2013</p>
<ul>
<li><b>Date, Time:</b> the date and time of the spray</li>
<li><b>Latitude, Longitude:</b> the Latitude and Longitude of the spray</li>
</ul>
<p><b>weather.csv</b> - weather data from 2007 to 2014. Column descriptions in noaa_weather_qclcd_documentation.pdf.</p>
<p><b>sampleSubmission.csv</b> - a sample submission file in the correct format</p>

In [2]:
test = pd.read_csv("../West Nile Virus/predict-west-nile-virus/test.csv")

spray_df = pd.read_csv("../West Nile Virus/predict-west-nile-virus/spray.csv")
weather_df = pd.read_csv("../West Nile Virus/predict-west-nile-virus/weather.csv")

print("Test size: ", test.shape)

Test size:  (116293, 11)


<b>Data Profiling and Exploratory Data Analysis (EDA)</b>

Let's look at what's in each data set using Pandas Profiling.<br> 
<https://www.kaggle.com/nulldata/intro-to-pandas-profiling-simple-fast-eda><br>
<https://en.wikipedia.org/wiki/Exploratory_data_analysis>

<p>Conda installs the 1.41 version of pandas profiling. Current is 2.0 and they haven't updated Conda yet. Beware that some tutorials aren't showing the used version in this project.</p>
<p>Reports are saved in the same folder as .ipynb file</p>

<b>Converting string/object Dates to datetime types</b><br>
<https://www.datacamp.com/community/tutorials/converting-strings-datetime-objects>
<p>train_df has a categorical field called Species with seven distinct values. We need to change this categorical feature to a binary one with the process called OneHotEncoding.</p>
<p>Let's also change the Date data type from object (string) to datetime64 in train_df, weather_df, and spray_df. This will allow us to use the datetime library on these values</p>

In [3]:
#creating dataframe with binary categorical dummy columns of each species of mosquito
testB = pd.get_dummies(test['Species'])


#putting new dataframe features into test dataframe  
for feature in testB:
    test[feature] = testB[feature]

spray_df['Date'] = pd.to_datetime(spray_df['Date'])
weather_df['Date'] = pd.to_datetime(weather_df['Date'])

test['Date'] = pd.to_datetime(test['Date'])

<b>Getting Rid of Duplicate Data</b><br>
<https://thispointer.com/pandas-find-duplicate-rows-in-a-dataframe-based-on-all-or-selected-columns-using-dataframe-duplicated-in-python/>
<p>Let's find the duplicate rows for each dataframe and drop them. Looking at the profiles for all three dataframes there seems to be exact duplicates in all of them. We will keep each first occurence and drop the rest. There is no duplicate data in testing.</p>

In [4]:
#duplicate rows except for the first occurence based on all features
spray_duplicate = spray_df[spray_df.duplicated()]
weather_duplicate = weather_df[weather_df.duplicated()]

#drop duplicates from dataframe
spray_df.drop(spray_duplicate.index, axis=0, inplace=True)
weather_df.drop(weather_duplicate.index, axis=0, inplace=True)

#print out change of sizes
print("Spray Duplicate Count: ", spray_duplicate.shape, "  Spray Size After Duplicate Removal: ", spray_df.shape)
print("Weather Duplicate Count: ", weather_duplicate.shape, " Weather Size After Duplicate Removal: ", weather_df.shape)

Spray Duplicate Count:  (541, 4)   Spray Size After Duplicate Removal:  (14294, 4)
Weather Duplicate Count:  (0, 22)  Weather Size After Duplicate Removal:  (2944, 22)


<b>Cleaning up Weather Dataframe</b><br><https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html><br><https://www.thoughtco.com/what-is-trace-of-precipitation-3444238>
<p>weather dataframe has 15 categorical features that need to be converted to numerical/binary</p>
<p>First feature Station needs to be made into binary features. Stations has only two outputs, either station 1 or station2.</p>
<p>Tavg, PrecipTotal, Depart, WetBulb, SnowFall, StnPressure, SeaLevel, Depth, AvgSpeed, Heat, Cool need to be floats.</p>
<p>There are 11 missing datas that have been converted to 0.0 floats in Tavg. To fill in Tavg, we're going to take its corresponding Tmin, Tmax and calculate the Tavg.</p>
<p>Water1 has nothing but missing data and needs to be dropped. There is over 50% data missing for feature CodeSum and it too should be dropped.</p>

In [5]:
#function that changes missing 'M' to 0.0, trace 'T' to '0.005', and anything else to data type float    
def convert_types(feature):
    weather_df[feature] = weather_df[feature].str.replace('M', '0.0')
    weather_df[feature] = weather_df[feature].str.replace('T', '0.005')
    weather_df[feature] = weather_df[feature].astype(float)

#features that represent number measurements but are object data types    
converted_features = ['Tavg', 'PrecipTotal', 'Depart', 'WetBulb', 'SnowFall', 'StnPressure', 'SeaLevel', 'Depth', 'AvgSpeed', 'Heat', 'Cool']    

#convert features
for element in converted_features:
    convert_types(element)

#change the 0.0 Tavg values and calculate a value using corresponding Tmin and Tmax    
weather_df['Tavg'][weather_df.Tavg == 0] = (weather_df['Tmin'] + weather_df['Tmax']) / 2    

weather_df.drop('CodeSum', axis=1, inplace=True)
weather_df.drop('Water1', axis=1, inplace=True)
weather_df.drop('Sunrise', axis=1, inplace=True)
weather_df.drop('Sunset', axis=1, inplace=True)

display(weather_df.columns)

C:\Users\Ashley\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Depth', 'SnowFall', 'PrecipTotal',
       'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

<b>Drop Address Related Features from Test Dataframe</b>
<p>We have the exact position with longitude and latitude for all data sets.</p>
<p>So Address, Block, Street, AddressNumberAndStreet, and AddressAccuracy can be dropped.</p>

In [6]:
test.drop(['Address','Block','Street','AddressNumberAndStreet','AddressAccuracy'], axis=1, inplace=True)

display(test.columns)

Index(['Id', 'Date', 'Species', 'Trap', 'Latitude', 'Longitude',
       'CULEX ERRATICUS', 'CULEX PIPIENS', 'CULEX PIPIENS/RESTUANS',
       'CULEX RESTUANS', 'CULEX SALINARIUS', 'CULEX TARSALIS',
       'CULEX TERRITANS', 'UNSPECIFIED CULEX'],
      dtype='object')

<b>Merge Test and Weather Dataframes</b><br><https://www.datacamp.com/community/tutorials/joining-dataframes-pandas>
<p>Let's make a unique key value so that we can merge train, weather, and spray dataframes.</p>

<b>Find the nearest Station (in Weather) for each trap location (in Test)</b><br><https://github.com/geopy/geopy><br><https://geopy.readthedocs.io/en/latest/#module-geopy.distance><br><br>
<b>Weather Data</b>

<p>It is believed that hot and dry conditions are more favorable for West Nile virus than cold and wet. We provide you with the dataset from NOAA of the weather conditions of 2007 to 2014, during the months of the tests.</p> 

<li>Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level</li>
<li>Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level</li>

<p>Station + Date is being used as a unique key to merging Train and Weather Dataframes</p>

In [8]:
#Longitude and Latitude values for station 1 and station 2
station1Lat = 41.995
station1Long = -87.933
station2Lat = 41.786
station2Long = -87.752

station1_location = (station1Lat, station1Long)
station2_location = (station2Lat, station2Long)

station_closest_test = []


#going through each longitude and latitude set and finding the difference in distance between each station then assigning one
#or two for station closest list
for index in test.index:
    trap_location = (test['Latitude'][index], test['Longitude'][index])
    if great_circle(trap_location, station1_location).miles < great_circle(trap_location, station2_location).miles:
        station_closest_test.append(1)
    else:
        station_closest_test.append(2)
        
#add to Station column and station closest values
test['Station'] = station_closest_test

#Have a unique key that lets Test and Weather merge on the same date and station
test['Key'] = test['Station'] + test['Date']
weather_df['Key'] = weather_df['Station'] + weather_df['Date']

test_weather_merge = pd.merge(test, weather_df, on='Key', how='inner')

test_weather_merge.drop(['Date_y', 'Station_y'], axis=1, inplace=True)

test_weather_merge.rename(columns={'Date_x':'Date', 'Station_x':'Station'}, inplace=True)

In [9]:
display(test_weather_merge.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 32 columns):
Id                        116293 non-null int64
Date                      116293 non-null datetime64[ns]
Species                   116293 non-null object
Trap                      116293 non-null object
Latitude                  116293 non-null float64
Longitude                 116293 non-null float64
CULEX ERRATICUS           116293 non-null uint8
CULEX PIPIENS             116293 non-null uint8
CULEX PIPIENS/RESTUANS    116293 non-null uint8
CULEX RESTUANS            116293 non-null uint8
CULEX SALINARIUS          116293 non-null uint8
CULEX TARSALIS            116293 non-null uint8
CULEX TERRITANS           116293 non-null uint8
UNSPECIFIED CULEX         116293 non-null uint8
Station                   116293 non-null int64
Key                       116293 non-null datetime64[ns]
Tmax                      116293 non-null int64
Tmin                      116293 non-null int64
T

None

<b>Spray Data</b>

<p>Each trap is affected by spraying based off time and distance. To create density the traps must be measured from the nearest spray point and then the time difference between when the trap is checked and when the area is spayed must occur. This allows us a change in density of how strongly a trap is affected by spraying.</p> 

<b>Finding A Relationship with Test_Weather_Merge Dataframe</b>
<p>Each row in this database shows the location and time of a trap that was checked. Some of these traps are near the location of spraying. Each row on the spray dataframe is of when (date and time) and location of the spray. Let's have an indication of if a trap is near a spray location, how far away, and when that spray occured.</p>
<p>Let's have some binary markers that indicate a general time and place relating the train data with the spray data. In the cell below each lat/long coordinates from each trap check is compared to all of the locations of the sprays. New features will be added to train_weather_merge that are binary features telling us a general location and time of a spray near each trap.</p>

In [10]:
# source: https://stackoverflow.com/a/29546836
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    #km = 6367 * c 
    m = 3957 * c
    return m

In [13]:
import time
start_time = time.time()

test['<= 90 days']  = 0
test['<= 183 days'] = 0
test['<= 365 days'] = 0
test['<= 730 days'] = 0
test['> 730 days'] = 0

test['<= 1/2 mile']  = 0
test['<= 1 mile'] = 0
test['<= 3 miles'] = 0
test['<= 5 miles'] = 0
test['> 5 miles'] = 0

#iterrating rows of test
for index in test.index:
    
    #for each latitude and longitude index a tuple of the two are made
    trap_location = (test['Latitude'][index], test['Longitude'][index])
    
    #we are taking the difference in test and spray time, converting it from timedelta to float
    #then taking that list and taking all of times that are over zero and putting it into a new list
    temp_time_list = (test['Date'][index] - spray_df['Date']).astype('timedelta64[D]') 
    temp_time_list_pruned = temp_time_list[temp_time_list>0]    
    
    #we are taking this new list of positive times and getting a list of booleans that will then be the values of our columns 
    test.at[index, '<= 90 days'] = int((temp_time_list_pruned <= 90).any()) 
    test.at[index, '<= 183 days'] = int((temp_time_list_pruned <= 183).any())
    test.at[index, '<= 365 days'] = int((temp_time_list_pruned <= 365).any())
    test.at[index, '<= 730 days'] = int((temp_time_list_pruned <= 730).any())
    test.at[index, '> 730 days'] = int((temp_time_list_pruned > 730).any())

    #each trap location is compaired to each latitude and longitude of each spray
    distance_list = haversine_np(spray_df['Longitude'],spray_df['Latitude'], trap_location[1], trap_location[0])
    
    #we are taking this list and creating a new one that has only the distances that correlate with our positive time list
    distance_list_pruned =  distance_list[temp_time_list>0]
    
    #this new list of distances of positive times creates a new list of booleans that are the values of our columns
    test.at[index, '<= 1/2 mile'] = int((distance_list_pruned <= 0.5).any())
    test.at[index, '<= 1 mile'] = int((distance_list_pruned <= 1).any())
    test.at[index, '<= 3 miles'] = int((distance_list_pruned <= 3).any())
    test.at[index, '<= 5 miles'] = int((distance_list_pruned <= 5).any())
    test.at[index, '> 5 miles'] = int((distance_list_pruned > 5).any())
    
    if index % 500 == 0: print(index)

#shows us the ellapsed time that it takes to run cell
elapsed_time = time.time() - start_time
print("Time: ", elapsed_time/60)

display(test)



0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
8500

,Id,Date,Species,Trap,Latitude,Longitude,CULEX ERRATICUS,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,...,<= 90 days,<= 183 days,<= 365 days,<= 730 days,> 730 days,<= 1/2 mile,<= 1 mile,<= 3 miles,<= 5 miles,> 5 miles
0,1,2008-06-11,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2008-06-11,CULEX RESTUANS,T002,41.954690,-87.800991,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,2008-06-11,CULEX PIPIENS,T002,41.954690,-87.800991,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,2008-06-11,CULEX SALINARIUS,T002,41.954690,-87.800991,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2008-06-11,CULEX TERRITANS,T002,41.954690,-87.800991,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116288,116289,2014-10-02,CULEX SALINARIUS,T054C,41.925652,-87.633590,0,0,0,0,...,0,0,0,1,1,0,0,1,1,1
116289,116290,2014-10-02,CULEX TERRITANS,T054C,41.925652,-87.633590,0,0,0,0,...,0,0,0,1,1,0,0,1,1,1
116290,116291,2014-10-02,CULEX TARSALIS,T054C,41.925652,-87.633590,0,0,0,0,...,0,0,0,1,1,0,0,1,1,1
116291,116292,2014-10-02,UNSPECIFIED CULEX,T054C,41.925652,-87.633590,0,0,0,0,...,0,0,0,1,1,0,0,1,1,1


<p>Droping duplicate Features 'Species' and 'Key'.</p>

In [14]:
test_final_df = pd.concat([test_weather_merge, test], axis=1)

test_final_df.drop('Species', axis=1, inplace=True)
test_final_df.drop('Key', axis=1, inplace=True)
#final_df.drop('geometry', axis=1, inplace=True)

<p>Taking categorical feature stations and making them binary. Then dropping stations.</p>

In [16]:
#creating dataframe with binary categorical dummy columns of each station
test_station_encoded_features = pd.get_dummies(test_final_df['Station'])
    
#putting new dataframe features into test dataframe
for feature in test_station_encoded_features:
    test_final_df[feature] = test_station_encoded_features[feature]
    
test_final_df.drop('Station', axis=1, inplace=True)

In [18]:
display(test_final_df.head(), test_final_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 52 columns):
Id                        116293 non-null int64
Date                      116293 non-null datetime64[ns]
Trap                      116293 non-null object
Latitude                  116293 non-null float64
Longitude                 116293 non-null float64
CULEX ERRATICUS           116293 non-null uint8
CULEX PIPIENS             116293 non-null uint8
CULEX PIPIENS/RESTUANS    116293 non-null uint8
CULEX RESTUANS            116293 non-null uint8
CULEX SALINARIUS          116293 non-null uint8
CULEX TARSALIS            116293 non-null uint8
CULEX TERRITANS           116293 non-null uint8
UNSPECIFIED CULEX         116293 non-null uint8
Tmax                      116293 non-null int64
Tmin                      116293 non-null int64
Tavg                      116293 non-null float64
Depart                    116293 non-null float64
DewPoint                  116293 non-null int64
WetBulb

,Id,Date,Trap,Latitude,Longitude,CULEX ERRATICUS,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,...,<= 90 days,<= 183 days,<= 365 days,<= 730 days,> 730 days,<= 1/2 mile,<= 1 mile,<= 3 miles,<= 5 miles,> 5 miles
0,1,2008-06-11,T002,41.95469,-87.800991,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2008-06-11,T002,41.95469,-87.800991,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2008-06-11,T002,41.95469,-87.800991,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,2008-06-11,T002,41.95469,-87.800991,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,5,2008-06-11,T002,41.95469,-87.800991,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


None

In [20]:
#drop duplicated columns
test_final_df = test_final_df.loc[:,~test_final_df.columns.duplicated()]

In [21]:
test_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 39 columns):
Id                        116293 non-null int64
Date                      116293 non-null datetime64[ns]
Trap                      116293 non-null object
Latitude                  116293 non-null float64
Longitude                 116293 non-null float64
CULEX ERRATICUS           116293 non-null uint8
CULEX PIPIENS             116293 non-null uint8
CULEX PIPIENS/RESTUANS    116293 non-null uint8
CULEX RESTUANS            116293 non-null uint8
CULEX SALINARIUS          116293 non-null uint8
CULEX TARSALIS            116293 non-null uint8
CULEX TERRITANS           116293 non-null uint8
UNSPECIFIED CULEX         116293 non-null uint8
Tmax                      116293 non-null int64
Tmin                      116293 non-null int64
Tavg                      116293 non-null float64
Depart                    116293 non-null float64
DewPoint                  116293 non-null int64
WetBulb

<b>Export final database to CSV File for Easy Access</b>

In [22]:
test_final_df.to_csv('test_final_df.csv', index=False)